In [1]:
%matplotlib inline
import pandas as pd

In [2]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime
from pyspark import SQLContext
sqlContext = SQLContext(sc)

import pyspark.sql.functions as F
import numpy as np

%matplotlib inline
import matplotlib
matplotlib.use('Agg')
#import matplotlib.pyplot
import matplotlib.pyplot as plt

/Users/l744679/Applications/anaconda/lib/python2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)
/Users/l744679/Applications/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [3]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

time: 3.51 ms


In [3]:
titles = pd.DataFrame.from_csv('data/titles.csv', index_col=None)
titles.head()

,title,year
0,A Lélek órása,1923
1,Aizaugusa gravi viegli krist,1986
2,Agliyorum,1988
3,0_1_0,2008
4,97 fung lau mung,1994


In [4]:
cast = pd.DataFrame.from_csv('data/cast.csv', index_col=None)
cast.head()

,title,year,name,type,character,n
0,The Core,2003,Alejandro Abellan,actor,U.S.S. Soldier,NaN
1,Il momento di uccidere,1968,Remo De Angelis,actor,Dago,9
2,Across the Divide,1921,Thomas Delmar,actor,Dago,4
3,Revan,2012,Diego James,actor,Dago,NaN
4,Un homme marche dans la ville,1950,Fabien Loris,actor,Dago,12


In [4]:
titles = sqlContext.read.parquet(os.path.expanduser('~/Dropbox/Projects/data/movielens_p/titles'))
titles.cache()
cast = sqlContext.read.parquet(os.path.expanduser('~/Dropbox/Projects/data/movielens_p/cast'))
cast.cache()

DataFrame[title: string, year: int, name: string, type: string, character: string, n: int]

time: 1.66 s


### Define a year as a "Superman year" whose films feature more Superman characters than Batman. How many years in film history have been Superman years?

In [18]:
sb_counts = cast[(cast.character == 'Superman') | (cast.character == 'Batman')]\
    .groupby(['year', 'character']).count()

time: 8.55 ms


In [36]:
sb_counts[(sb_counts.character == 'Superman')].select('year', 'count').withColumnRenamed('count', 'superman')\
    .join(sb_counts[(sb_counts.character == 'Batman')].select('year', 'count').withColumnRenamed('count', 'batman'), on='year', how='outer')\
    .fillna(0)\
    .filter('superman > batman')\
    .show()

+----+--------+------+
|year|superman|batman|
+----+--------+------+
|1948|       1|     0|
|1950|       1|     0|
|1951|       2|     0|
|1954|       5|     0|
|1955|       1|     0|
|1973|       2|     1|
|1978|       2|     0|
|1980|       2|     0|
|1983|       2|     0|
|1987|       1|     0|
|2006|       1|     0|
+----+--------+------+

time: 2.09 s


### How many years have been "Batman years", with more Batman characters than Superman characters?

In [37]:
sb_counts[(sb_counts.character == 'Superman')].select('year', 'count').withColumnRenamed('count', 'superman')\
    .join(sb_counts[(sb_counts.character == 'Batman')].select('year', 'count').withColumnRenamed('count', 'batman'), on='year', how='outer')\
    .fillna(0)\
    .filter('superman < batman')\
    .show()

+----+--------+------+
|year|superman|batman|
+----+--------+------+
|1938|       0|     1|
|1940|       0|     1|
|1943|       0|     1|
|1949|       0|     2|
|1953|       0|     2|
|1957|       0|     2|
|1965|       0|     1|
|1966|       0|     1|
|1967|       0|     1|
|1988|       0|     1|
|1989|       0|     1|
|1993|       0|     2|
|1998|       0|     1|
|1999|       0|     1|
|2004|       0|     1|
|2005|       0|     1|
|2009|       1|     3|
|2011|       2|     4|
|2012|       0|     2|
|2013|       0|     4|
+----+--------+------+
only showing top 20 rows

time: 2.15 s


### Plot the number of actor roles each year and the number of actress roles each year over the history of film.

In [16]:
cast.groupby(['year', 'type']).count().sort(['year', 'type']).show()

+----+-------+-----+
|year|   type|count|
+----+-------+-----+
|1894|  actor|    2|
|1894|actress|    1|
|1899|  actor|    6|
|1900|  actor|    2|
|1905|  actor|    1|
|1906|  actor|   14|
|1906|actress|    3|
|1907|  actor|    5|
|1908|  actor|   29|
|1908|actress|   13|
|1909|  actor|    3|
|1910|  actor|   51|
|1910|actress|   26|
|1911|  actor|  136|
|1911|actress|   64|
|1912|  actor|  241|
|1912|actress|  148|
|1913|  actor|  654|
|1913|actress|  344|
|1914|  actor| 2215|
+----+-------+-----+
only showing top 20 rows

time: 375 ms


### Plot the number of actor roles each year and the number of actress roles each year, but this time as a kind='area' plot.

### Plot the difference between the number of actor roles each year and the number of actress roles each year over the history of film.

In [ ]:
counts = cast.groupby(['year', 'type']).count()

In [20]:
unstack(counts, 'year', 'type', 'count').fillna(0).selectExpr('*', 'actor - actress as diff').show()

+----+-----+-------+----+
|year|actor|actress|diff|
+----+-----+-------+----+
|1894|    2|      1|   1|
|1899|    6|      0|   6|
|1900|    2|      0|   2|
|1905|    1|      0|   1|
|1906|   14|      3|  11|
|1907|    5|      0|   5|
|1908|   29|     13|  16|
|1909|    3|      0|   3|
|1910|   51|     26|  25|
|1911|  136|     64|  72|
|1912|  241|    148|  93|
|1913|  654|    344| 310|
|1914| 2215|   1142|1073|
|1915| 3404|   2001|1403|
|1916| 4839|   2833|2006|
|1917| 5201|   3047|2154|
|1918| 4924|   2800|2124|
|1919| 4915|   2788|2127|
|1920| 5264|   2888|2376|
|1921| 4276|   2403|1873|
+----+-----+-------+----+
only showing top 20 rows

time: 12.8 s


### Plot the fraction of roles that have been 'actor' roles each year in the hitsory of film.

In [ ]:
counts = cast.groupby(['year', 'type']).count()

In [21]:
unstack(counts, 'year', 'type', 'count').fillna(0).selectExpr('*', 'actor / (actor + actress) as actor_pct').show()

+----+-----+-------+------------------+
|year|actor|actress|         actor_pct|
+----+-----+-------+------------------+
|1894|    2|      1|0.6666666666666666|
|1899|    6|      0|               1.0|
|1900|    2|      0|               1.0|
|1905|    1|      0|               1.0|
|1906|   14|      3|0.8235294117647058|
|1907|    5|      0|               1.0|
|1908|   29|     13|0.6904761904761905|
|1909|    3|      0|               1.0|
|1910|   51|     26|0.6623376623376623|
|1911|  136|     64|              0.68|
|1912|  241|    148|0.6195372750642674|
|1913|  654|    344| 0.655310621242485|
|1914| 2215|   1142|0.6598153112898422|
|1915| 3404|   2001|0.6297872340425532|
|1916| 4839|   2833|0.6307351407716372|
|1917| 5201|   3047|0.6305771096023278|
|1918| 4924|   2800|0.6374935266701192|
|1919| 4915|   2788|0.6380630923017007|
|1920| 5264|   2888|0.6457311089303238|
|1921| 4276|   2403|0.6402156011378949|
+----+-----+-------+------------------+
only showing top 20 rows

time: 12.5 s


### Plot the fraction of supporting (n=2) roles that have been 'actor' roles each year in the history of film.

In [5]:
sb_counts = cast[cast.n == 2].groupby(['year', 'type']).count()
sb_counts.show()

+----+-------+-----+
|year|   type|count|
+----+-------+-----+
|1987|  actor|  693|
|1960|  actor|  458|
|1933|  actor|  395|
|1906|  actor|    2|
|2002|actress|  911|
|1975|actress|  429|
|1948|actress|  358|
|1921|actress|  348|
|1998|  actor|  893|
|1971|  actor|  625|
|1944|  actor|  345|
|1917|  actor|  604|
|2013|actress| 1604|
|1986|actress|  502|
|1959|actress|  452|
|1932|actress|  456|
|2009|  actor| 1859|
|1982|  actor|  528|
|1955|  actor|  395|
|1928|  actor|  378|
+----+-------+-----+
only showing top 20 rows

time: 4.85 s


In [13]:
sb_counts[sb_counts.type == 'actor'].selectExpr('year', 'count as actor')\
    .join(sb_counts.groupby('year').sum('count').withColumnRenamed('sum(count)', 'sum'), on='year', how='outer')\
    .selectExpr('*', 'actor / sum as actor_pct')\
    .show()

+----+-----+---+-------------------+
|year|actor|sum|          actor_pct|
+----+-----+---+-------------------+
|1899|    2|  2|                1.0|
|1906|    2|  3| 0.6666666666666666|
|1907|    1|  1|                1.0|
|1908|    2|  2|                1.0|
|1910|    1|  3| 0.3333333333333333|
|1911|   14| 20|                0.7|
|1912|    9| 24|              0.375|
|1913|   64|115| 0.5565217391304348|
|1914|  171|325| 0.5261538461538462|
|1915|  334|629| 0.5310015898251192|
|1916|  518|905| 0.5723756906077349|
|1917|  604|998| 0.6052104208416834|
|1918|  563|912| 0.6173245614035088|
|1919|  537|915| 0.5868852459016394|
|1920|  509|895| 0.5687150837988827|
|1921|  390|738| 0.5284552845528455|
|1922|  372|712| 0.5224719101123596|
|1923|  297|577| 0.5147313691507799|
|1924|  320|654| 0.4892966360856269|
|1925|  337|716|0.47067039106145253|
+----+-----+---+-------------------+
only showing top 20 rows

time: 8.25 s


In [7]:
def unstack(df, i_col, p_col, v_col):
    cats = df.select(p_col).distinct().sort(p_col).toPandas()[p_col].tolist()
    for i in range(len(cats)):
        if i == 0:
            wide = df[(df[p_col] == cats[i])].select(i_col, v_col).withColumnRenamed(v_col, cats[i])
        else:
            wide = wide.join(df[(df[p_col] == cats[i])].select(i_col, v_col).withColumnRenamed(v_col, cats[i]), on=i_col, how='outer')
    return wide

sb_counts_w = unstack(sb_counts, 'year', 'type', 'count').fillna(0)
sb_counts_w.show()

+----+-----+-------+
|year|actor|actress|
+----+-----+-------+
|1899|    2|      0|
|1906|    2|      1|
|1907|    1|      0|
|1908|    2|      0|
|1910|    1|      2|
|1911|   14|      6|
|1912|    9|     15|
|1913|   64|     51|
|1914|  171|    154|
|1915|  334|    295|
|1916|  518|    387|
|1917|  604|    394|
|1918|  563|    349|
|1919|  537|    378|
|1920|  509|    386|
|1921|  390|    348|
|1922|  372|    340|
|1923|  297|    280|
|1924|  320|    334|
|1925|  337|    379|
+----+-----+-------+
only showing top 20 rows

time: 11.6 s


In [8]:
sb_counts_w.withColumn('actor_pct', sb_counts_w.actor / (sb_counts_w.actor + sb_counts_w.actress)).show()

+----+-----+-------+-------------------+
|year|actor|actress|          actor_pct|
+----+-----+-------+-------------------+
|1899|    2|      0|                1.0|
|1906|    2|      1| 0.6666666666666666|
|1907|    1|      0|                1.0|
|1908|    2|      0|                1.0|
|1910|    1|      2| 0.3333333333333333|
|1911|   14|      6|                0.7|
|1912|    9|     15|              0.375|
|1913|   64|     51| 0.5565217391304348|
|1914|  171|    154| 0.5261538461538462|
|1915|  334|    295| 0.5310015898251192|
|1916|  518|    387| 0.5723756906077349|
|1917|  604|    394| 0.6052104208416834|
|1918|  563|    349| 0.6173245614035088|
|1919|  537|    378| 0.5868852459016394|
|1920|  509|    386| 0.5687150837988827|
|1921|  390|    348| 0.5284552845528455|
|1922|  372|    340| 0.5224719101123596|
|1923|  297|    280| 0.5147313691507799|
|1924|  320|    334| 0.4892966360856269|
|1925|  337|    379|0.47067039106145253|
+----+-----+-------+-------------------+
only showing top

### Build a plot with a line for each rank n=1 through n=3, where the line shows what fraction of that rank's roles were 'actor' roles for each year in the history of film.

In [25]:
sb_counts = cast[cast.n.between(1, 3)].groupby(['year', 'n', 'type']).count().sort(['year', 'n', 'type'])
sb_counts.show()

+----+---+-------+-----+
|year|  n|   type|count|
+----+---+-------+-----+
|1899|  1|  actor|    2|
|1899|  2|  actor|    2|
|1906|  1|  actor|    2|
|1906|  1|actress|    1|
|1906|  2|  actor|    2|
|1906|  2|actress|    1|
|1906|  3|actress|    1|
|1907|  1|  actor|    1|
|1907|  2|  actor|    1|
|1907|  3|  actor|    1|
|1908|  1|  actor|    1|
|1908|  2|  actor|    2|
|1908|  3|  actor|    2|
|1909|  1|  actor|    1|
|1910|  1|  actor|    2|
|1910|  1|actress|    3|
|1910|  2|  actor|    1|
|1910|  2|actress|    2|
|1910|  3|  actor|    2|
|1911|  1|  actor|   17|
+----+---+-------+-----+
only showing top 20 rows

time: 344 ms


In [27]:
def unstack(df, i_col, p_col, v_col):
    cats = df.select(p_col).distinct().sort(p_col).toPandas()[p_col].tolist()
    for i in range(len(cats)):
        if i == 0:
            wide = df[(df[p_col] == cats[i])].drop(p_col).withColumnRenamed(v_col, cats[i])
        else:
            wide = wide.join(df[(df[p_col] == cats[i])].drop(p_col).withColumnRenamed(v_col, cats[i]), on=i_col, how='outer')
    return wide

sb_counts_w = unstack(sb_counts, ['year', 'n'], 'type', 'count').fillna(0)
sb_counts_w.show()

+----+---+-----+-------+
|year|  n|actor|actress|
+----+---+-----+-------+
|1912|  3|   14|      5|
|2004|  1| 1642|    651|
|1977|  1|  761|    286|
|2004|  2| 1152|    996|
|1950|  1|  604|    268|
|1977|  2|  558|    415|
|2004|  3| 1211|    784|
|1923|  1|  360|    245|
|1950|  2|  416|    401|
|1977|  3|  567|    343|
|1923|  2|  297|    280|
|1950|  3|  488|    304|
|1923|  3|  395|    167|
|2015|  1| 2751|   1269|
|1988|  1| 1021|    382|
|2015|  2| 2145|   1529|
|1961|  1|  714|    297|
|1988|  2|  720|    543|
|2015|  3| 2106|   1325|
|1934|  1|  572|    311|
+----+---+-----+-------+
only showing top 20 rows

time: 47.3 s


In [29]:
sb_counts_w.sort(['year', 'n']).selectExpr('*', 'actor / (actor + actress) as actor_pct').show()

+----+---+-----+-------+------------------+
|year|  n|actor|actress|         actor_pct|
+----+---+-----+-------+------------------+
|1899|  1|    2|      0|               1.0|
|1899|  2|    2|      0|               1.0|
|1906|  1|    2|      1|0.6666666666666666|
|1906|  2|    2|      1|0.6666666666666666|
|1906|  3|    0|      1|               0.0|
|1907|  1|    1|      0|               1.0|
|1907|  2|    1|      0|               1.0|
|1907|  3|    1|      0|               1.0|
|1908|  1|    1|      0|               1.0|
|1908|  2|    2|      0|               1.0|
|1908|  3|    2|      0|               1.0|
|1909|  1|    1|      0|               1.0|
|1910|  1|    2|      3|               0.4|
|1910|  2|    1|      2|0.3333333333333333|
|1910|  3|    2|      0|               1.0|
|1911|  1|   17|      9|0.6538461538461539|
|1911|  2|   14|      6|               0.7|
|1911|  3|   15|      3|0.8333333333333334|
|1912|  1|   15|     12|0.5555555555555556|
|1912|  2|    9|     15|        